# Анализ базы данных клиентов приложения для чтения книг.

## Цель проекта

Целью проекта является анализ базы данных мобильного приложения, представляющего собой сервис для чтения книг по подписке.
Результаты исследования помогут сформулировать ценностное предложение для нового продукта.

## Задачи проекта

В этом исследовании требуется выполнить следующие задачи:
* посчитать, сколько книг вышло после 1 января 2000 года;
* для каждой книги посчитайте количество обзоров и среднюю оценку;
* определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Описание данных

Для исследования предоставлена база данных со следующими таблицами: **books, authors, publishers, ratings, reviews**. 

Таблица **books** cодержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

Таблица **authors** cодержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

Таблица **publishers** cодержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

Таблица **ratings** cодержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

Таблица **reviews** cодержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

### Схема данных.

<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png" width="600" height="300">

## Результаты

Импортируем библиотеки, которые используются в этом проекте.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

Зададим параметры подключения к базе данных.

In [2]:
# устанавливаем параметры для подключения к БД

# параметры подключения удалены в версии для github
# db_config = {'user': '', # имя пользователя
#'pwd': '', # пароль
#'host': '',
#'port': , # порт подключения
#'db': ''} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Напишем функцию get_sql_results() для вывода результатов запроса к базе данных.

In [3]:
def get_sql_results(query_text, connector):
    """Функция  get_sql_results на вход принимает 2 параметра: query_text - текстовый запрос к бд, connector - коннектор к бд.
    Возвращает результаты запроса в виде датафрейма pandas"""
    # создаем подключение к БД
    con=connector.connect()
    # чтобы выполнить SQL-запрос, используем Pandas
    return pd.io.sql.read_sql(sql=text(query), con = con)

Выведем первые 5 строк  каждой таблицы.

In [4]:
for table_name in ('books', 'authors', 'ratings', 'reviews', 'publishers'):
    query = f'''SELECT * 
               FROM {table_name} 
               LIMIT 5'''
    print(f'Выведем первые 5 строк таблицы {table_name}')
    display(get_sql_results(query, engine))

Выведем первые 5 строк таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем первые 5 строк таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем первые 5 строк таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем первые 5 строк таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Выведем первые 5 строк таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Проверим пропуски и дубликаты в таблицах базы данных.

In [5]:
for table_name in ('books', 'authors', 'ratings', 'reviews', 'publishers'):
    query = f'''SELECT * 
               FROM {table_name}'''
    temp = get_sql_results(query, engine)
    print(f'Число пропусков в таблице: {table_name}')
    display(temp.isna().sum())
    print(f'Число дубликатов в таблице: {table_name}')
    display(temp.duplicated().sum())

Число пропусков в таблице: books


book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

Число дубликатов в таблице: books


0

Число пропусков в таблице: authors


...

0

Число пропусков в таблице: publishers


publisher_id    0
publisher       0
dtype: int64

Число дубликатов в таблице: publishers


0

Пропуски и дубликаты в таблицах отсутствуют.

**1)** Посчитаем, сколько книг вышло после 1 января 2000 г.

In [6]:
query = '''SELECT COUNT(book_id) 
           FROM books
           WHERE  publication_date > '2000-01-01' '''

books_cnt = get_sql_results(query, engine)
books_cnt

,count
0,819


После 1 января 2000 г. вышло 819 книг, которые содержатся в базе данных сервиса.

**2)** Выведем число обзоров и среднюю оценку для каждой книги сервиса.

**Версия 1**

In [7]:
query = '''SELECT b.title
                  , COUNT(rev.review_id) AS reviews_count
                  , AVG(rat.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rev on b.book_id = rev.book_id
           LEFT JOIN ratings AS rat on b.book_id = rat.book_id
           GROUP BY b.title'''
books_rating_reviews = get_sql_results(query, engine)
books_rating_reviews

,title,reviews_count,avg_rating
0,The Count of Monte Cristo,115,4.217391
1,Count Zero (Sprawl #2),4,2.500000
2,The Botany of Desire: A Plant's-Eye View of th...,4,3.500000
3,The Poisonwood Bible,110,4.363636
4,The Canterbury Tales,18,3.333333
...,...,...,...
994,Of Love and Other Demons,4,4.500000
995,In the Heart of the Sea: The Tragedy of the Wh...,9,3.333333
996,Welcome to Temptation (Dempseys #1),4,5.000000
997,World's End (The Sandman #8),4,4.500000


**Версия 2**

In [8]:
query = '''SELECT b.book_id, b.title
                  , COUNT (DISTINCT(rev.review_id)) AS reviews_count
                  , AVG(rat.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rev on b.book_id = rev.book_id
           LEFT JOIN ratings AS rat on b.book_id = rat.book_id
           GROUP BY b.book_id
           ORDER BY reviews_count DESC'''
books_rating_reviews = get_sql_results(query, engine)
books_rating_reviews

,book_id,title,reviews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Выведем полученную таблицу еще раз с сортировкой по убыванию среднего рейтинга каждой книги.

In [10]:
books_rating_reviews.sort_values(by='avg_rating', ascending=False)

,book_id,title,reviews_count,avg_rating
940,967,Welcome to Temptation (Dempseys #1),2,5.00
300,553,School's Out—Forever (Maximum Ride #2),3,5.00
943,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
709,418,March,2,5.00
710,421,Marvel 1602,2,5.00
...,...,...,...,...
497,915,The World Is Flat: A Brief History of the Twen...,3,2.25
685,371,Junky,2,2.00
510,202,Drowning Ruth,3,2.00
659,316,His Excellency: George Washington,2,2.00


Всего в базе данных приложения 1000 книг, наибольшее число обзоров (7) - у книги "Twilight (Twilight #1)". Средний рейтинг книг изменяется от 1.5 до 5.

**3)** Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [11]:
query = ''' SELECT p.publisher_id, p.publisher, COUNT(b.book_id)
            FROM books AS b 
            LEFT JOIN publishers AS p on b.publisher_id=p.publisher_id
            WHERE book_id IN ((SELECT book_id
                               FROM books
                               WHERE num_pages > 50))
            GROUP BY p.publisher_id, p.publisher
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1;'''
publisher_max_books = get_sql_results(query, engine)
publisher_max_books

,publisher_id,publisher,count
0,212,Penguin Books,42


В базе данных сервиса больше всего книг толще 50 страниц издательства Penguin Books - 42 книги.

**4)** Определим автора с самой высокой средней оценкой книг.

In [12]:
query = '''SELECT a.author, AVG(rat.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN ratings AS rat on b.book_id = rat.book_id
           LEFT JOIN authors AS a on b.author_id=a.author_id
           WHERE b.book_id IN (SELECT b.book_id
                               FROM books AS b
                               LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
                               GROUP BY b.book_id
                               HAVING COUNT(rat.rating_id) >= 50)
          GROUP BY a.author
          ORDER BY avg_rating DESC
          LIMIT 1'''
author_max_rating = get_sql_results(query, engine)
author_max_rating

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг (учитываются книги с 50 оценками и более) - J.K. Rowling и Mary GrandPré 🧙‍♂️.

**5)** Посчитаем среднее количество обзоров от пользователей, которые поставили более 48 оценок.

In [13]:
query = '''WITH cnt_rev AS (SELECT username, COUNT(review_id) AS cnt_reviews
                            FROM reviews
                            WHERE username IN (SELECT username
                                               FROM ratings 
                                               GROUP BY username
                                               HAVING COUNT(rating_id) > 48)
                            GROUP BY username)
                                               
           SELECT AVG(cnt_reviews)
           FROM cnt_rev;'''
avg_cnt_reviews = get_sql_results(query, engine)
avg_cnt_reviews

,avg
0,24.0


У пользователей, которые поставили больше 48 оценок, в среднем, 24 обзора.

## Общий вывод.

В этом проекте анализировалась база данных мобильного приложения-старпата, представляющего собой сервис для чтения книг по подписке.
Заказчик предоставил базу данных с 5 таблицами:
* books - с данными о книгах
* authors - с данными об авторах
* publishers - с данными об издательствах
* rating - с данными о пользовательских оценках 
* reviews - с данными о пользовательских обзорах. 

В табликах пропусков и дубликатов не обнаружено.

После анализа базы данных получились следующие результаты:

* в базе данных сервиса содержится 819 книг, вышедших после 1 января 2000 г.
* всего в базе данных приложения 1000 книг, наибольшее число обзоров (7) - у книги "Twilight (Twilight #1)". Средний рейтинг книг изменяется от 1.5 до 5.
* в базе данных сервиса больше всего книг толще 50 страниц издательства Penguin Books - 42 книги.
* автор с самой высокой средней оценкой книг (учитывались книги с 50 оценками и более) - J.K. Rowling и Mary GrandPré 🧙‍♂️.
* у пользователей, которые поставили больше 48 оценок, в среднем, 24 обзора.